In [42]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [43]:
trainft = pd.read_csv('train_fix.csv')
testft = pd.read_csv('test_fix.csv')
trainorg = pd.read_csv('train_org.csv')

In [44]:
print('The shape of train features is:', trainft.shape)
print('The shape of test features is:', testft.shape)

The shape of train features is: (91531, 27)
The shape of test features is: (27859, 27)


In [45]:
trainft = pd.get_dummies(trainft)
testft = pd.get_dummies(testft)
labels = np.array(trainorg['is_canceled'])
adrlabels = np.array(trainorg['adr'])
trainft = trainft.drop('market_segment_Undefined', axis = 1)
trainft = trainft.drop('distribution_channel_Undefined', axis = 1)
print(trainft.shape)
print(testft.shape)

(91531, 41)
(27859, 41)


In [46]:
feature_list = list(trainft.columns)
print(feature_list)
feature_list = list(testft.columns)
print(feature_list)

['stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'booking_changes', 'required_car_parking_spaces', 'total_of_special_requests', 'total_guests', 'total_nights', 'arrival_date_month_number', 'hotel_type', 'country_type', 'reserved_room_type_categorical', 'is_family', 'long_stay', 'is_weekend', 'meal_numerical', 'is_agent', 'is_company', 'lead_time', 'agent', 'company', 'market_segment_Aviation', 'market_segment_Complementary', 'market_segment_Corporate', 'market_segment_Direct', 'market_segment_Groups', 'market_segment_Offline TA/TO', 'market_segment_Online TA', 'distribution_channel_Corporate', 'distribution_channel_Direct', 'distribution_channel_GDS', 'distribution_channel_TA/TO', 'deposit_type_No Deposit', 'deposit_type_Non Refund', 'deposit_type_Refundable', 'customer_type_Contract', 'customer_type_Group', 'customer_type_Transient', 'customer_type_Transient-Party']
['stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', '

In [47]:
trainft = np.array(trainft)
testft = np.array(testft)

In [48]:
train_features, test_features, train_labels, test_labels = train_test_split(
    trainft, labels, test_size=0.25, random_state=42)
trainadr_features, testadr_features, trainadr_labels, testadr_labels = train_test_split(
    trainft, adrlabels, test_size=0.25, random_state=42)

In [49]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [51]:
from xgboost import XGBRegressor
xgb_adr = XGBRegressor()
xgb_adr.fit(trainadr_features, trainadr_labels)
predict_adr = xgb_adr.predict(testadr_features)
xgbRMSE = rmse(predict_adr, testadr_labels)
print("rmse is: " + str(xgbRMSE))

rmse is: 27.508291475263672


In [52]:
testpredict_xgb = xgb_adr.predict(testft)
adrpredict_xgb = pd.DataFrame(testpredict_xgb,columns=['predict_adr'])
adrpredict_xgb.to_csv('adr_predict.csv')

In [53]:
from xgboost import XGBClassifier
xgb_cancel = XGBClassifier()
xgb_cancel.fit(train_features, train_labels)
predict_cancel = xgb_cancel.predict(test_features)
xgberr = abs(predict_cancel - test_labels)
print("error is: " + str(np.sum(xgberr) / np.size(xgberr)))

[19:57:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
error is: 0.123934798758904


In [54]:
cancelpredict_xgb = xgb_cancel.predict(testft)
predict_xgb = pd.DataFrame(cancelpredict_xgb,columns=['is_canceled'])
predict_xgb.to_csv('xgb_cancelpredict.csv')